In [ ]:
# !pip install pyblp

# Pure Logit using PyBLP

The question is that, in the Cereal industry, what if we halved an important product's price?

In [1]:
import pyblp
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
# np.set_printoptions(suppress=True)

In [2]:
pyblp.options.digits = 3
pyblp.options.verbose = False
pd.options.display.precision = 3
pd.options.display.max_columns = 50
pd.options.display.float_format = '{:.5f}'.format

import IPython.display
IPython.display.display(IPython.display.HTML('<style>pre { white-space: pre !important; }</style>'))

In [3]:
address = 'https://raw.githubusercontent.com/Mixtape-Sessions/Demand-Estimation/main/Exercises/Data/products.csv'
cereal = pd.read_csv(address)

## 1. Describing Data

The data contains information about 24 breakfast cereals across 94 markets. Each row is a product-market pair. Each market has the same set of breakfast cereals, although with different prices and quantities.

The data were motivated by real cereal purchase data across 47 US cities in the first 2 quarters of 1988. So, each city, quarter is considered a market in the data. So, C01Q1 is city 1 in quarter 1.

Also, the product column is the firm, product case. We have five firms each producing 1 to 9 brands.

In [101]:
cereal.sample(1)

,market,product,mushy,servings_sold,city_population,price_per_serving,price_instrument
2039,C56Q1,F6B18,0,1668905.44197,364040,0.11636,0.09725


And some summary stats for the data is as follows:

In [102]:
float_columns = cereal.select_dtypes(include=['float', 'int'])
string_columns = cereal.select_dtypes(include='object')

float_stats = float_columns.describe().loc[['mean', 'std']]
string_stats = string_columns.nunique()
float_stats

,mushy,servings_sold,city_population,price_per_serving,price_instrument
mean,0.33333,1202962.11926,641995.02128,0.12574,0.09081
std,0.47151,3986557.61837,1161489.03885,0.02904,0.02936


In [103]:
string_stats

market     94
product    24
dtype: int64

So, as explained before, we have 94 markets and 24 products.

## 2. Compute market shares

To transform observed quantities into market shares, we first need to define a market size. We'll assume that the potential number of servings sold in a market is the city's **total population multiplied by 90 days in the quarter**.

In [4]:
cereal['market_size'] = cereal['city_population'] * 90
cereal['market_share'] = cereal['servings_sold']/cereal['market_size']
cereal['outside_share'] = 1 - cereal.groupby('market')['market_share'].transform('sum')

In [5]:
cereal[['outside_share', 'market_share']].describe().loc[['mean', 'std', 'min', 'max']]

,outside_share,market_share
mean,0.52420,0.01983
std,0.10962,0.02560
min,0.30458,0.00018
max,0.81517,0.44688


## 3. Estimate the pure logit model with OLS

In the pure logit case $u_{ijt} = \delta_{jt} + \underbrace{\mu_{ijt}}_{=0} + \epsilon_{ijt}$, the market share is:
$$
s_{jt} = \frac{\exp{\delta_{jt}}}{1+\sum_{k\in\mathcal{J}_t} \exp{\delta_{kt}}} \,\, ,\,\, s_{0t} = \frac{1}{1+\sum_{k\in\mathcal{J}_t} \exp{\delta_{kt}}}
$$
Therefore:
$$
\log\left(\frac{s_{jt}}{s_{0t}}\right) = \delta_{jt} = x_{jt} \beta - \alpha p_{jt} + \xi_{jt}
$$

LHS is known from the data. Therefore, one can solve for the parameter estimates using OLS (disregarding the endogeneity of prices for now).

In [46]:
model = smf.ols(formula='logit_delta ~ 1 + mushy + price_per_serving', data=cereal)
result = model.fit(cov_type='HC0')
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:            logit_delta   R-squared:                       0.034
Model:                            OLS   Adj. R-squared:                  0.033
Method:                 Least Squares   F-statistic:                     39.89
Date:                Tue, 20 Aug 2024   Prob (F-statistic):           9.48e-18
Time:                        17:22:01   Log-Likelihood:                -3583.2
No. Observations:                2256   AIC:                             7172.
Df Residuals:                    2253   BIC:                             7190.
Df Model:                           2                                         
Covariance Type:                  HC0                                         
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            -2.9345      0.10

According to this result, a dollor increase in the price would decrease the mean utility of the product by $-7.48$. To get more meaningful, this change would lead to a 748% decrease in the proportional ratio of the market share of that product. This means that, demand is very elastic to prices. Also, the coefficient of mushy is positive (though, insignificent). 

## 4. Run the same regression with PyBLP

Here, we want to use PyBLP to solve for the model. First, we need to rename some columns.

In [6]:
cereal.rename(columns={'market': 'market_ids'}, inplace=True)
cereal.rename(columns={'product': 'product_ids'}, inplace=True)
cereal.rename(columns={'market_share': 'shares'}, inplace=True)
cereal.rename(columns={'price_per_serving': 'prices'}, inplace=True)

By default, PyBLP treats prices as endogenous, so it won't include them in its matrix of instruments. But the "instruments" for running an OLS regression are the same as the full set of regressors. So when running an OLS regression and not account for price endogeneity, we'll "instrument" for prices with prices themselves. We can do this by creating a new column demand_instruments0 equal to prices. PyBLP will recognize all columns that start with demand_instruments and end with 0, 1, 2, etc., as "excluded" instruments to be stacked with the exogenous characteristics to create the full set of instruments.

In [7]:
cereal['demand_instruments0'] = cereal['prices']

In [8]:
ols_problem = pyblp.Problem(pyblp.Formulation('1 + mushy + prices'), cereal)
print(ols_problem)

Dimensions:
 T    N     K1    MD 
---  ----  ----  ----
94   2256   3     3  

Formulations:
     Column Indices:         0     1      2   
--------------------------  ---  -----  ------
X1: Linear Characteristics   1   mushy  prices


So, in the model here, we have 94 markets, a total 2256 data points for all products, 3 exogenous variables and 3 instruments. As we have no endogeneity assumption in the prices, instruments and the exogenous variables are the same.

In [9]:
ols_results = ols_problem.solve(method='1s')
print(ols_results)

Problem Results Summary:
GMM   Objective  Clipped  Weighting Matrix  Covariance Matrix
Step    Value    Shares   Condition Number  Condition Number 
----  ---------  -------  ----------------  -----------------
 1    +3.47E-24     0        +1.40E+03          +1.30E+03    

Cumulative Statistics:
Computation   Objective 
   Time      Evaluations
-----------  -----------
 00:00:00         1     

Beta Estimates (Robust SEs in Parentheses):
     1          mushy       prices   
-----------  -----------  -----------
 -2.93E+00    +7.48E-02    -7.48E+00 
(+1.08E-01)  (+5.41E-02)  (+8.40E-01)


So, the result is the same as above. Solve in PyBLP would use HC0 standard errors. Also, the method 1s used here means that we only use 1 step GMM. So, it is just a simple OLS regression here.

## 5. Add market and product fixed effects

Here in the data, we have multiple products for each market and variations in prices both within and between markets. Also, as expected, we think that prices are endogenous and correlated with $\xi_{jt}$. Therefore, we need to do something about it. For this part, suppose that the structure of error term is $\xi_{jt} = \xi_j + \xi_t + \Delta\xi_{jt}$. Also, suppose that most of the correlation of prices and error term is due to market and product error terms. Then, adding fixed effects of these both terms would help to reduce the bias of price coefficient.

We can use built-in function $C$ in PyBLP for this purpose. Also, we are using absorbing and not actually adding all dummy variables to the model.

In [10]:
FE_problem = pyblp.Problem(pyblp.Formulation('0 + prices', 
                                                 absorb='C(market_ids) + C(product_ids)'), cereal)
print(FE_problem)

Dimensions:
 T    N     K1    MD    ED 
---  ----  ----  ----  ----
94   2256   1     1     2  

Formulations:
     Column Indices:          0   
--------------------------  ------
X1: Linear Characteristics  prices


In [11]:
FE_results = FE_problem.solve(method='1s')
print(FE_results)

Problem Results Summary:
GMM   Objective  Clipped  Weighting Matrix
Step    Value    Shares   Condition Number
----  ---------  -------  ----------------
 1    +3.49E-32     0        +1.00E+00    

Cumulative Statistics:
Computation   Objective 
   Time      Evaluations
-----------  -----------
 00:00:00         1     

Beta Estimates (Robust SEs in Parentheses):
  prices   
-----------
 -2.86E+01 
(+8.92E-01)


So, adding product and market FEs, the price coefficient has risen drastically to $-28.6$. ***This means that, we have had upward bias in the price coefficient. Therefore, we can say that prices are positively correlated with unobserved quality.***

## 6. Add an instrument for price

Given the same structure for the error term as above ($\xi_{jt} = \xi_j + \xi_t + \Delta\xi_{jt}$), we controled for the FEs to control for correlations of prices with the first two terms. Yet, to also control for the thrid, we need to add some instrument.

The data comes with a column price_instrument that we should interpret as a valid instrument for price that satisfies the needed exclusion restriction. It could be a cost-shifter, a valid Hausman instrument, or similar.

Before using it, we should first run a first-stage regression to make sure that it's a relevant instrument for price. So, we will regress prices on the instrument and the FEs.

In [12]:
model = smf.ols(formula='prices ~ 0 + price_instrument + C(market_ids) + C(product_ids)',
                data=cereal)
result = model.fit(cov_type='HC0')
summary_df = result.summary2().tables[1]

# Reorder to have 'price_instrument' first
cols_order = ['price_instrument'] + [col for col in summary_df.index if col != 'price_instrument']
ordered_summary_df = summary_df.loc[cols_order]

# Print the reordered summary
print(ordered_summary_df)

                          Coef.  Std.Err.         z   P>|z|  [0.025  0.975]
price_instrument        0.87710   0.00658 133.23203 0.00000 0.86420 0.89000
C(market_ids)[C01Q1]    0.03732   0.00118  31.71476 0.00000 0.03502 0.03963
C(market_ids)[C01Q2]    0.03892   0.00140  27.77837 0.00000 0.03617 0.04166
C(market_ids)[C03Q1]    0.04071   0.00111  36.72759 0.00000 0.03854 0.04288
C(market_ids)[C03Q2]    0.03756   0.00122  30.74272 0.00000 0.03517 0.03996
...                         ...       ...       ...     ...     ...     ...
C(product_ids)[T.F3B14] 0.01090   0.00089  12.20351 0.00000 0.00915 0.01265
C(product_ids)[T.F4B02] 0.01330   0.00093  14.26615 0.00000 0.01147 0.01513
C(product_ids)[T.F4B10] 0.00342   0.00078   4.41200 0.00001 0.00190 0.00494
C(product_ids)[T.F4B12] 0.00578   0.00083   6.97063 0.00000 0.00415 0.00740
C(product_ids)[T.F6B18] 0.00732   0.00088   8.33941 0.00000 0.00560 0.00904

[118 rows x 6 columns]


So, the instrument is actually correlated with prices. So, the test of the first stage is done here.

In [13]:
cereal['demand_instruments0'] = cereal['price_instrument']

FEIV_problem = pyblp.Problem(pyblp.Formulation('0 + prices', 
                                                 absorb='C(market_ids) + C(product_ids)'), cereal)
print(FEIV_problem)

Dimensions:
 T    N     K1    MD    ED 
---  ----  ----  ----  ----
94   2256   1     1     2  

Formulations:
     Column Indices:          0   
--------------------------  ------
X1: Linear Characteristics  prices


In [15]:
FEIV_results = FEIV_problem.solve(method='1s')
print(FEIV_results)

Problem Results Summary:
GMM   Objective  Clipped  Weighting Matrix
Step    Value    Shares   Condition Number
----  ---------  -------  ----------------
 1    +4.47E-29     0        +1.00E+00    

Cumulative Statistics:
Computation   Objective 
   Time      Evaluations
-----------  -----------
 00:00:00         1     

Beta Estimates (Robust SEs in Parentheses):
  prices   
-----------
 -3.06E+01 
(+9.68E-01)


So, adding the IV, now the coefficient is $-30.6$. Therefore, the correlation between prices and $\Delta\xi_{ij}$ was causing an upward bias. Therefore, this correlation was positive.

All results are gathered in the following table:

In [20]:
pd.DataFrame(index=FE_results.beta_labels, data={
    ("Estimates", "OLS"): ols_results.beta[-1:].flat,
    ("Estimates", "+FE"): FE_results.beta.flat,
    ("Estimates", "+IV"): FEIV_results.beta.flat,
    ("SEs", "OLS"): ols_results.beta_se[-1:].flat,
    ("SEs", "+FE"): FE_results.beta_se.flat,
    ("SEs", "+IV"): FEIV_results.beta_se.flat,
})

Estimates                         SEs                
             OLS       +FE       +IV     OLS     +FE     +IV
prices  -7.48014 -28.61787 -30.59952 0.83954 0.89195 0.96784

## 7. Cut a price in half and see what happens

Now, it is time to do some counterfactual analysis. We'll select a single market, the most recent quarter in the first city. We'll pretend that we're firm one, and deciding whether we want to cut the price of our brand four's product. 

In [28]:
counterfactual_market = 'C01Q2'
counterfactual_data = cereal.loc[cereal['market_ids'] == counterfactual_market,
                                 ['product_ids', 'mushy', 'prices', 'shares']]

counterfactual_data['new_prices'] = counterfactual_data['prices']
counterfactual_data.loc[counterfactual_data['product_ids'] == 'F1B04', 'new_prices'] /= 2

In [29]:
counterfactual_data['new_shares'] = FEIV_results.compute_shares(
    prices = counterfactual_data['new_prices'], 
    market_id=counterfactual_market)
counterfactual_data['share_changes%'] = 100 * (counterfactual_data['new_shares']
                                               - counterfactual_data['shares']) / counterfactual_data['shares']
counterfactual_data[['product_ids', 'shares', 'new_shares', 'share_changes%']]

,product_ids,shares,new_shares,share_changes%
24,F1B04,0.00644,0.02085,223.63766
25,F1B06,0.14128,0.13923,-1.45019
26,F1B07,0.08789,0.08662,-1.45019
27,F1B09,0.00662,0.00653,-1.45019
28,F1B11,0.05427,0.05349,-1.45019
29,F1B13,0.02198,0.02166,-1.45019
30,F1B17,0.01055,0.01039,-1.45019
31,F1B30,0.00131,0.00129,-1.45019
32,F1B45,0.01052,0.01037,-1.45019
33,F2B05,0.05907,0.05821,-1.45019


So, by halving the price of the fourth product of firm 1, the share of this product has risen by 223%. Also, the percentage change of all other products is the same. This is because the cross elasticities of shares in the pure logit model is $-\alpha P_{F1B04} S_{F1B04}$. In other words, those cross elasticities do not depend on the features of that product and only depending on the product which we are trying to find the sensitivity to. This is a big caveat of this model, representing unsual and unrealistic estimates for elasticities. **Cannibalization estimates don't seem reasonable -- we'd expect more cannibalization from the other products of firm one that are more similar to the product whose price is being cut**. 

## 8. Compute demand elasticities

Now, we want to estimate the demand elasticities. These measure what the model predicts will happen to demand in percentage terms when there's a 1% change in price of a product. So, for the counterfactual market we will solve for this matrix. Digaonal elements are own elasticities and off-diagonal elements are cross-elasticities.

In [27]:
demand_elas_counterfactual_market = FEIV_results.compute_elasticities(
    name='prices', market_id=counterfactual_market)
pd.DataFrame(demand_elas_counterfactual_market)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,-2.36280,0.60974,0.19548,0.01560,0.27737,0.06210,0.04968,0.00603,0.04743,0.17973,0.04765,0.01740,0.07606,0.01849,0.02641,0.07382,0.00560,0.00279,0.03041,0.02531,0.01879,0.02272,0.02628,0.01373
1,0.01532,-3.70603,0.19548,0.01560,0.27737,0.06210,0.04968,0.00603,0.04743,0.17973,0.04765,0.01740,0.07606,0.01849,0.02641,0.07382,0.00560,0.00279,0.03041,0.02531,0.01879,0.02272,0.02628,0.01373
2,0.01532,0.60974,-2.02857,0.01560,0.27737,0.06210,0.04968,0.00603,0.04743,0.17973,0.04765,0.01740,0.07606,0.01849,0.02641,0.07382,0.00560,0.00279,0.03041,0.02531,0.01879,0.02272,0.02628,0.01373
3,0.01532,0.60974,0.19548,-2.33978,0.27737,0.06210,0.04968,0.00603,0.04743,0.17973,0.04765,0.01740,0.07606,0.01849,0.02641,0.07382,0.00560,0.00279,0.03041,0.02531,0.01879,0.02272,0.02628,0.01373
4,0.01532,0.60974,0.19548,0.01560,-4.83302,0.06210,0.04968,0.00603,0.04743,0.17973,0.04765,0.01740,0.07606,0.01849,0.02641,0.07382,0.00560,0.00279,0.03041,0.02531,0.01879,0.02272,0.02628,0.01373
5,0.01532,0.60974,0.19548,0.01560,0.27737,-2.76326,0.04968,0.00603,0.04743,0.17973,0.04765,0.01740,0.07606,0.01849,0.02641,0.07382,0.00560,0.00279,0.03041,0.02531,0.01879,0.02272,0.02628,0.01373
6,0.01532,0.60974,0.19548,0.01560,0.27737,0.06210,-4.66053,0.00603,0.04743,0.17973,0.04765,0.01740,0.07606,0.01849,0.02641,0.07382,0.00560,0.00279,0.03041,0.02531,0.01879,0.02272,0.02628,0.01373
7,0.01532,0.60974,0.19548,0.01560,0.27737,0.06210,0.04968,-4.59599,0.04743,0.17973,0.04765,0.01740,0.07606,0.01849,0.02641,0.07382,0.00560,0.00279,0.03041,0.02531,0.01879,0.02272,0.02628,0.01373
8,0.01532,0.60974,0.19548,0.01560,0.27737,0.06210,0.04968,0.00603,-4.45869,0.17973,0.04765,0.01740,0.07606,0.01849,0.02641,0.07382,0.00560,0.00279,0.03041,0.02531,0.01879,0.02272,0.02628,0.01373
9,0.01532,0.60974,0.19548,0.01560,0.27737,0.06210,0.04968,0.00603,0.04743,-2.86299,0.04765,0.01740,0.07606,0.01849,0.02641,0.07382,0.00560,0.00279,0.03041,0.02531,0.01879,0.02272,0.02628,0.01373


The non-realistic substitution patterns we saw in our counterfactual also show up here: all cross-price elasticities in each column are the same, even though we'd expect some differences for more similar products. Also, the own elasticities are actually large, suggesting a high substitution of products. For example, for the first product, a 1 percent increase in price would lead to a more than 2 percent decrease in demand. 

# Supplemented Questions

## 1. Try different standard errors

By default PyBLP would solve for the model using heteroskedastic standard errors. But, what if the error terms are actually clustered. Are error terms correlated across markets for a given product $j$ or across products for a given market $t$. This will lead to different results of standard errors.

In [22]:
cereal['demand_instruments0'] = cereal['price_instrument']
cereal['clustering_ids'] = cereal['product_ids']

FEIV_cluster_product = pyblp.Problem(pyblp.Formulation('0 + prices', 
                                                 absorb='C(market_ids) + C(product_ids)'), cereal)

FEIV_cluster_product_results = FEIV_cluster_product.solve(method='1s', 
                                                            se_type='clustered')

cereal['clustering_ids'] = cereal['market_ids']
FEIV_cluster_market = pyblp.Problem(pyblp.Formulation('0 + prices', 
                                                 absorb='C(market_ids) + C(product_ids)'), cereal)

FEIV_cluster_market_results = FEIV_cluster_market.solve(method='1s', 
                                                            se_type='clustered')

In [23]:
pd.DataFrame(index=FEIV_results.beta_labels, data={
    ("Estimates", "+IV"): FEIV_results.beta.flat,
    ("SEs", "+IV"): FEIV_results.beta_se.flat,
    ("SEs", "+IV_cluster_product"): FEIV_cluster_product_results.beta_se.flat,
    ("SEs", "+IV_cluster_market"): FEIV_cluster_market_results.beta_se.flat,
})

Estimates     SEs                                       
             +IV     +IV +IV_cluster_product +IV_cluster_market
prices -30.59952 0.96784             1.18244            1.09257

So, clustering has risen the standard errors a little bit. 

## 2. Compute confidence intervals for your counterfactual

When we estimate the coefficients of the model, PyBLP will also solve for standard errors and CIs. Yet, when running counterfactuals, we only have point estimates. We would like to have some sense of how much these point estimates are valid. So, we need to come up with some CIs. We can do this using "***parametric bootstrap***" method.